In [29]:
import psycopg2
from psycopg2 import sql
import pandas as pd

In [30]:
def create_db(dbname):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
            database='postgres',
            user='postgres',
            password='password',
            host='localhost',
            port='5432'
        )
        
        connection.autocommit = True
        cursor = connection.cursor()
        
        # Create database using psycopg2.sql to safely construct the SQL statement
        cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(dbname)))
        
        print(f"Database {dbname} created successfully.")

    except Exception as e:
        print(f"Operation failed: {e}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def drop_db(dbname):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
            database='postgres',
            user='postgres',
            password='password',
            host='localhost',
            port='5432'
        )
        
        connection.autocommit = True
        cursor = connection.cursor()
        
        # Create database using psycopg2.sql to safely construct the SQL statement
        cursor.execute(sql.SQL("DROP DATABASE {}").format(sql.Identifier(dbname)))
        
        print(f"Database {dbname} dropped successfully.")

    except Exception as e:
        print(f"Operation failed: {e}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def transition(query, params=None, dbname='krannet'):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
        database=dbname,
        user='postgres',
        password='password',
        host='localhost',
        port= '5432')
        
        cursor = connection.cursor()

        # Transition
        cursor.execute(sql.SQL('BEGIN'))

        if not params:
            cursor.execute(sql.SQL(query))
        else: 
            cursor.execute(sql.SQL(query), params)
        
        # Commit transition
        connection.commit()

        print("Transaction committed successfully.")

    except Exception as e:
        connection.rollback()
        print(f"Transaction failed: {e}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def fetch_table_data(table_name, dbname='krannet'):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
        database=dbname,
        user='postgres',
        password='password',
        host='localhost',
        port= '5432')
        
        # Fetch data using pandas
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, connection)
        
        return df

    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return None

    finally:
        if connection:
            connection.close()

Some usefull SQL prommt:
- `CREATE database <db_name>`
- `DROP database <db_name>`

## Create DB

In [31]:
# Env Variables
DB_NAME = 'krannet'

# Create DB
create_db(DB_NAME)
#drop_db(DB_NAME)

Operation failed: database "krannet" already exists



## Create outer table

In [32]:
query = f""" 
CREATE TABLE IF NOT EXISTS outer_table (
id SERIAL PRIMARY KEY,
title VARCHAR NOT NULL,
year CHAR(4)
)
"""

transition(query)

Transaction committed successfully.


In [40]:
# insert data into outher_table

titles = ['meeting1', 'meeting2']
years = [1990, 1990]

for t, y in zip(titles, years):
    query = 'INSERT INTO outer_table (title, year) VALUES (%s, %s)'
    params = (t, y)
    transition(query, params)

Transaction committed successfully.
Transaction committed successfully.


In [41]:
df = fetch_table_data(table_name='outer_table')
df.head()

/var/folders/nv/ljsvt6pd7v526ds10294vq_w00011d/T/ipykernel_14414/2259759351.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,id,title,year
0,1,meeting1,1990
1,2,meeting2,1990


## Create inner table

In [ ]:
# create inner table with speaker and n. of interventions
create_inner_table = f""" 
CREATE TABLE IF NOT EXISTS outer_table (
id SERIAL PRIMARY KEY,
speaker VARCHAR(50) NOT NULL,
interventions SMALLINT NOT NULL
)
"""

# TODO - Create the inner table to be connected to the outer table 
# MAYBE connecting id